In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from google.colab import drive
import pandas as pd
import os

# 1. Mount Google Drive
# You will be prompted to permit access to your Drive files
drive.mount('/content/drive')

# 2. Define the path to your file
# Note: In Colab, the root of your Drive is '/content/drive/My Drive/'
folder_path = '/content/drive/My Drive/WIA1007/'
file_path = os.path.join(folder_path, 'carlist_scraped_data.csv')

# 3. Load the dataset
try:
    df = pd.read_csv(file_path)
    print("Dataset loaded successfully!")

    # Display the first 5 rows to verify
    display(df.head())

except FileNotFoundError:
    print(f"Error: The file was not found at: {file_path}")
    print("Please ensure the folder 'WIA1007' exists in your 'My Drive' root.")


# 2. Data Cleaning & Preprocessing
def clean_currency(val):
    try:
        if pd.isna(val): return np.nan
        return float(str(val).replace('RM', '').replace(',', '').strip())
    except: return np.nan

def clean_monthly(val):
    try:
        if pd.isna(val): return np.nan
        return float(str(val).replace('RM', '').replace('/ month', '').replace(',', '').strip())
    except: return np.nan

def clean_mileage(val):
    try:
        if pd.isna(val): return np.nan
        s = str(val).upper().replace(',', '').strip()
        if '-' in s:
            parts = s.replace('K KM', '').replace('KM', '').split('-')
            avg = (float(parts[0].strip()) + float(parts[1].strip())) / 2
            return avg * 1000 if 'K' in s else avg
        num = float(s.replace('K KM', '').replace('KM', '').strip())
        return num * 1000 if 'K' in s else num
    except: return np.nan

# Create numeric versions for calculations
df['Price_Num'] = df['List_Price'].apply(clean_currency)
df['Monthly_Num'] = df['Monthly_Installment'].apply(clean_monthly)
df['Mileage_Num'] = df['Mileage'].apply(clean_mileage)

# Define variables and their properties
variables = {
    'Description': {'type': 'Categorical', 'level': 'Nominal', 'unit': 'N/A'},
    'Monthly_Installment': {'type': 'Numerical', 'level': 'Ratio', 'unit': 'RM', 'num_col': 'Monthly_Num'},
    'List_Price': {'type': 'Numerical', 'level': 'Ratio', 'unit': 'RM', 'num_col': 'Price_Num'},
    'Model': {'type': 'Categorical', 'level': 'Nominal', 'unit': 'N/A'},
    'Mileage': {'type': 'Numerical', 'level': 'Ratio', 'unit': 'KM', 'num_col': 'Mileage_Num'},
    'Gear_Type': {'type': 'Categorical', 'level': 'Nominal', 'unit': 'N/A'},
    'Location': {'type': 'Categorical', 'level': 'Nominal', 'unit': 'N/A'}
}

# 3. Helper Functions
def get_outliers(series):
    if series.dtype == 'object': return 'N/A'
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return ((series < lower) | (series > upper)).sum()

def format_freq(col_name):
    # Handle specific formatting for Location and Mileage strings
    if col_name == 'Location':
        data = df[col_name].str.split(',').str[-1].str.strip()
    elif col_name == 'Mileage':
        data = df[col_name].str.replace(' KM', '')
    else:
        data = df[col_name]

    counts = data.value_counts(normalize=True) * 100

    if col_name == 'Gear_Type':
        return ", ".join([f"{k}: {v:.1f}%" for k, v in counts.items()])

    top3 = counts.head(3)
    others_val = counts.iloc[3:].sum()

    if col_name in ['Model', 'Mileage', 'Location']:
        res = ", ".join([f"{k}: {v:.1f}%" for k, v in top3.items()])
        return f"{res}, Others: {others_val:.1f}%"
    else:
        top3_sum = top3.sum()
        return f"Top 3: {top3_sum:.1f}%, Others: {others_val:.1f}%"

# 4. Generate Table 1: Data Properties
table1_list = []
for var, props in variables.items():
    series = df[props.get('num_col', var)]
    orig_series = df[var]

    table1_list.append({
        'Variable': var,
        'Types of Data': props['type'],
        'Data Types': str(orig_series.dtype),
        'Measurement Level': props['level'],
        'Units': props['unit'],
        'Range': f"{series.min()} - {series.max()}" if props['type'] == 'Numerical' else 'N/A',
        'Min Value': series.min() if props['type'] == 'Numerical' else 'N/A',
        'Top Value': orig_series.mode()[0],
        'Unique Values': orig_series.nunique(),
        'Null Values': orig_series.isnull().sum(),
        'Outliers': get_outliers(series)
    })

table1 = pd.DataFrame(table1_list)

# 5. Generate Table 2: Statistics
table2_list = []
for var, props in variables.items():
    series = df[props.get('num_col', var)]
    orig_series = df[var]

    is_num = props['type'] == 'Numerical'

    table2_list.append({
        'Variable': var,
        'Frequency': format_freq(var),
        'Percentile (25th, 50th, 75th)': f"{series.quantile(0.25):.1f}, {series.quantile(0.5):.1f}, {series.quantile(0.75):.1f}" if is_num else 'N/A',
        'Data Completeness': f"{(orig_series.count()/len(df))*100:.2f}%",
        'Mean': f"{series.mean():.2f}" if is_num else 'N/A',
        'Median': f"{series.median():.2f}" if is_num else 'N/A',
        'Mode': orig_series.mode()[0],
        'Std Dev': f"{series.std():.2f}" if is_num else 'N/A',
        'Variance': f"{series.var():.2f}" if is_num else 'N/A',
        'Skewness': f"{series.skew():.2f}" if is_num else 'N/A',
        'Kurtosis': f"{series.kurtosis():.2f}" if is_num else 'N/A'
    })

table2 = pd.DataFrame(table2_list)

# Display Results
print("--- Table 1: Data Properties ---")
display(table1)
print("\n--- Table 2: Statistics ---")
display(table2)

# Correctly Save to Google Drive
t1_save_path = os.path.join(folder_path, 'Table1_Properties.csv')
t2_save_path = os.path.join(folder_path, 'Table2_Statistics.csv')

table1.to_csv(t1_save_path, index=False)
table2.to_csv(t2_save_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset loaded successfully!


,Description,Monthly_Installment,List_Price,Model,Mileage,Gear_Type,Location
0,2021 Toyota Hilux 2.4 E Dual Cab Pickup - UNDE...,RM 983 / month,"RM 75,800",Toyota Hilux,115 - 120K KM,Automatic,"Kuala Lumpur, Batu Caves / Selayang"
1,2019 Toyota Vios 1.5 G Sedan - (A) FACELIFT NS...,RM 607 / month,"RM 46,800",Toyota Vios,55 - 60K KM,Automatic,"Selangor, Seri Kembangan"
2,2013 Nissan Almera 1.5 E Sedan - impul,RM 231 / month,"RM 17,790",Nissan Almera,155 - 160K KM,Automatic,"Selangor, Banting"
3,2017 Perodua Myvi 1.5 AV Hatchback - Adv,RM 425 / month,"RM 32,800",Perodua Myvi,90 - 95K KM,Automatic,"Kuala Lumpur, Setapak"
4,2018/2019 BMW X5 2.0 xDrive40e M Sport SUV - Y...,"RM 1,283 / month","RM 99,000",BMW X5,70 - 75K KM,Automatic,"Selangor, Petaling Jaya"


--- Table 1: Data Properties ---


,Variable,Types of Data,Data Types,Measurement Level,Units,Range,Min Value,Top Value,Unique Values,Null Values,Outliers
0,Description,Categorical,object,Nominal,N/A,N/A,N/A,2018 Proton Preve 1.6 CFE Premium Sedan - Full...,3472,0,N/A
1,Monthly_Installment,Numerical,object,Ratio,RM,62.0 - 15298.0,62.0,RM 516 / month,1038,0,278
2,List_Price,Numerical,object,Ratio,RM,4800.0 - 1180000.0,4800.0,"RM 39,800",1301,1,278
3,Model,Categorical,object,Nominal,N/A,N/A,N/A,Perodua Myvi,307,0,N/A
4,Mileage,Numerical,object,Ratio,KM,2500.0 - 237214000.0,2500.0,85 - 90K KM,460,0,438
5,Gear_Type,Categorical,object,Nominal,N/A,N/A,N/A,Automatic,2,0,N/A
6,Location,Categorical,object,Nominal,N/A,N/A,N/A,"Selangor, Petaling Jaya",109,0,N/A



--- Table 2: Statistics ---


,Variable,Frequency,"Percentile (25th, 50th, 75th)",Data Completeness,Mean,Median,Mode,Std Dev,Variance,Skewness,Kurtosis
0,Description,"Top 3: 1.3%, Others: 98.7%",N/A,100.00%,N/A,N/A,2018 Proton Preve 1.6 CFE Premium Sedan - Full...,N/A,N/A,N/A,N/A
1,Monthly_Installment,"Top 3: 2.6%, Others: 97.4%","472.0, 738.0, 1242.0",100.00%,1064.05,738.00,RM 516 / month,1136.18,1290902.62,4.91,38.51
2,List_Price,"Top 3: 2.6%, Others: 97.4%","36400.0, 56888.0, 95800.0",99.98%,82074.61,56888.00,"RM 39,800",87639.24,7680636156.76,4.91,38.51
3,Model,"Perodua Myvi: 6.1%, Honda City: 5.0%, Honda Ci...",N/A,100.00%,N/A,N/A,Perodua Myvi,N/A,N/A,N/A,N/A
4,Mileage,"85 - 90K: 6.5%, 80 - 85K: 6.2%, 90 - 95K: 6.0%...","62500.0, 87500.0, 112500.0",100.00%,9518749.81,87500.00,85 - 90K KM,31884257.42,1016605871358311.62,3.65,13.36
5,Gear_Type,"Automatic: 97.8%, Manual: 2.2%",N/A,100.00%,N/A,N/A,Automatic,N/A,N/A,N/A,N/A
6,Location,"Cheras: 8.6%, Petaling Jaya: 6.4%, Johor Bahru...",N/A,100.00%,N/A,N/A,"Selangor, Petaling Jaya",N/A,N/A,N/A,N/A
